In [32]:
from elasticsearch import Elasticsearch, helpers
from dotenv import load_dotenv
import os
import datasets
load_dotenv(dotenv_path='elasticsearch.env')
load_dotenv(dotenv_path='use_auth_token.env')
username = os.getenv("username") 
password = os.getenv('password')
USE_AUTH_TOKEN = os.getenv('USE_AUTH_TOKEN')

# 엘라스틱서치 연결

In [20]:
es = Elasticsearch(['https://final-project.es.us-east4.gcp.elastic-cloud.com'], port=9243, http_auth=(username, password))

In [21]:
es.ping()

True

In [68]:
# 인덱스 삭제
es.indices.delete(index='summarization', ignore=[400, 404])

{'acknowledged': True}

In [ ]:
# 인덱스 보기
res = es.indices.get_alias("*")
for Name in res:
    print(Name)

# 허깅페이스 데이터셋 로드

In [58]:
train_dataset = datasets.load_dataset('metamong1/summarization', use_auth_token=USE_AUTH_TOKEN, split='train')
test_dataset = datasets.load_dataset('metamong1/summarization', use_auth_token=USE_AUTH_TOKEN, split='test')
validation_dataset = datasets.load_dataset('metamong1/summarization', use_auth_token=USE_AUTH_TOKEN, split='validation')

Reusing dataset summarization (/opt/ml/.cache/huggingface/datasets/metamong1___summarization/Summarization/1.0.0/b7b9872b62b80260483449a5850c3d67183f4bff5922d808dca247085f550838)
Reusing dataset summarization (/opt/ml/.cache/huggingface/datasets/metamong1___summarization/Summarization/1.0.0/b7b9872b62b80260483449a5850c3d67183f4bff5922d808dca247085f550838)
Reusing dataset summarization (/opt/ml/.cache/huggingface/datasets/metamong1___summarization/Summarization/1.0.0/b7b9872b62b80260483449a5850c3d67183f4bff5922d808dca247085f550838)


In [59]:
total_dataset = datasets.concatenate_datasets([train_dataset, test_dataset, validation_dataset])

In [60]:
train_dataset

Dataset({
    features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
    num_rows: 275219
})

In [61]:
total_dataset

Dataset({
    features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
    num_rows: 458699
})

# Elasticsearch 세팅 및 업로드 데이터

In [97]:
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer'
        }
      },
    }
  },
  'mappings':{
    'properties':{
      'title':{
        'type':'text',
        "analyzer": "nori_analyzer"
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      },
      'doc_type':{
        'type':'text',
      }
    }
  }
}

In [99]:
es.indices.create(index='summarization_nori', body=CONFIG)

/tmp/ipykernel_3156/1409656848.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='summarization_nori', body=CONFIG)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'summarization_nori'}

In [101]:
def generator(dataset):
    for i, data in enumerate(dataset):
        yield {
            '_index': 'summarization_nori',
            '_type': '_doc',
            '_id': i,
            '_source': {
                'title': data.get('title', ''),
                'text': data.get('text', ''),
                'doc_type': data.get('doc_type', '')
            }
        }
    print(i)
    raise StopIteration

In [102]:
rest_dataset = total_dataset[-198:]
gen_total = generator(total_dataset)
gen_rest = generator(rest_dataset)

In [103]:
try:
    res = helpers.bulk(es, gen_total, chunk_size = 500, request_timeout=60)
except Exception as e:
    print(e)
    print('Done')

/opt/conda/envs/final/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


458698
generator raised StopIteration
Done


In [104]:
es.count(index='summarization_nori')

{'count': 458500,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [43]:
try:
    res = helpers.bulk(es, gen_rest, chunk_size = 1)
except Exception as e:
    print(e)
    print('Done')

In [121]:
result = es.search(index='summarization_nori', body={'size':3, 'query':{'match':{'text':'''특혜 의혹을 받는 대장동 개발사업의 주무 부서장을 맡아 수사 기관의 조사를 받아오다 어제(21일) 숨진채 발견된숨진 채 성남도시개발공사(이하 성남도개공) 김문기 개발1처장의 유족은 오늘 "고인은 실무자였을 뿐"이라며 억울함을 주장했습니다.

김 처장의 동생 A씨는 오늘 김 처장의 빈소가 마련된 성남시 분당서울대병원 장례식장에서 기자회견을 열어 "부서장이라고 하더라도 위에 결정권자 없이는 (사업을 추진할) 힘이 없다"며 이같이 말했습니다.

김 처장이 숨지기 하루 전 함께 점심을 먹었다고 밝힌 A씨는 "당시 형에게 밥을 떠먹여 줘야 했을 정도로 몸 상태가 매우 좋지 않았다"고 전했습니다.
'''}}})['hits']['hits'][0]['_source']['title']



# 송영길 "김건희, 尹에 반말하는 실세"→ 野 "김혜경도 이재명에 반말, 트집도"

/tmp/ipykernel_3156/2846053363.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index='summarization_nori', body={'size':3, 'query':{'match':{'text':'''특혜 의혹을 받는 대장동 개발사업의 주무 부서장을 맡아 수사 기관의 조사를 받아오다 어제(21일) 숨진채 발견된숨진 채 성남도시개발공사(이하 성남도개공) 김문기 개발1처장의 유족은 오늘 "고인은 실무자였을 뿐"이라며 억울함을 주장했습니다.


In [122]:
result

'靑인사 `안 보인다` 지적에...뒤늦게 사망 특감반원 장례식장 찾은 김조원 민정수석'